In [1]:
%%capture
%run 05_block.ipynb
%run 06_mine.ipynb

Account state holds the current balance and nonce of an account.

In [21]:
class AccountState:
    def __init__(self): self.balance,self.nonce = 0,0
    def inc_nonce  (self):        self.nonce   += 1
    def add_balance(self, value): self.balance += value
    def sub_balance(self, value): self.balance -= value
    def json(self): return json.dumps(self.__dict__)
    def __str__(self): return f'balance: {self.balance}\nnonce: {self.nonce}'

The state stores the dictionary `state` which stores the `AccountState` for each address. A genesis block is used to initialize state the first time.

The state is updated with txs. `apply` validates if the tx is correct and then updates each `AccountState` accordingly.

In [22]:
class State:
    def __init__(self): self.state={}
    def new   (self, pub): self.state[pub] = AccountState()
    def is_new(self, pub): return pub not in self.state
    def add(self, pubs):
        for pub in pubs:
            if self.is_new(pub): self.new(pub)
    
    def init(self, gb):
        gh = gb.header
        assert gh.number    == 0
        assert gh.prev_hash == '0x0'
        for tx in gb.txs.values():
            self.add([tx.fr, tx.to])
            assert val_sig(tx)
            self.state[tx.fr].inc_nonce()
            self.state[tx.to].add_balance(tx.value)
        
    def apply(self, tx, miner):
        self.add([tx.fr,tx.to,miner])
        if tx.fr != tx.to: 
            assert val_sig(tx)
            assert self.state[tx.fr].nonce  == tx.nonce
            assert self.state[tx.fr].balance - tx.value > 0
        else: 
            assert tx.fr == tx.to == miner
        self.state[tx.fr].inc_nonce()
        self.state[tx.fr].sub_balance(tx.value)
        self.state[tx.to].add_balance(tx.value)
        self.state[miner].add_balance(tx.fee)
        return True
    
    def apply_reward(self, miner, reward):
        self.state[miner].add_balance(reward)
    
    def __getitem__(self, key): return self.state[key] 
    def __str__(self):
        return '\n'.join(f'{ph(k)}\n{v}\n' for k,v in self.state.items())

## Blockchain

Consists of a number of blocks. Each block points to the block that came before it. Imagine a linked list of blocks linked by its `prev_hash`.

A `candidate` block is one that could be used for mining.

In [23]:
class Blockchain:
    def __init__(self, gb):
        self.state  = State()
        self.blocks = []
        self.state.init(gb)
        self.blocks.append(gb)

    def candidate(self, txs):
        mt = MerkleTree(txs)
        bh = Header(mt.root, self.blocks[-1].hash, len(self.blocks), len(txs))
        return Block(bh, txs)
        
    def add(self, mb):
        assert self.val(mb)
        for tx in mb.txs.values(): assert self.state.apply(tx, mb.header.miner)
        self.state.apply_reward(mb.header.miner, mb.header.reward)
        self.blocks.append(mb)
        return True
    
    def val(self, mb):
        bh = mb.header
        assert val_pow(mb)
        assert bh.number == len(self.blocks)
        if len(self.blocks) > 0: assert self.blocks[-1].hash == bh.prev_hash
        return True
    
    def __getitem__(self, key): return self.state.__getitem__(key)
    def __str__(self): 
        return (('\n'*2+'--'*20+'\n'*2).join(f'{str(block)}' for block in self.blocks)
                +'\n'*2+'-'*6+'\nstate:\n'+'-'*6
                +'\n'+str(self.state))

In [24]:
acc1,acc2,acc3 = Account(),Account(),Account()
miner = Miner(acc3)

In [25]:
tx1 = acc1.signed_tx(acc2, 24, 0.1)
tx2 = acc2.signed_tx(acc1, 18, 0.12)

gb = mine_genesis([tx1, tx2])

In [27]:
bc = Blockchain(gb); print(bc)

root:          📤 0x7e04e96571...ee3
hash:          🕟 0xf9a0ba7c86...223
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          1617571974.650272 eth
volume:        42.0 eth
fees:          0.22 eth

txs:
💳 0x4D4d -> 🕗 0xf19d 24.0 eth
🕗 0xf19d -> 💳 0x4D4d 18.0 eth

------
state:
------
💳 0x4D4d017Ea9...444
balance: 18.0
nonce: 1

🕗 0xf19dB46598...b3C
balance: 24.0
nonce: 1



In [28]:
tx1 = acc1.signed_tx(acc2, 3, 0.05)
tx2 = acc2.signed_tx(acc1, 9, 0.16)

block = bc.candidate([tx1,tx2])
mb    = miner.mine([tx1,tx2], gb.header, 2, 50)
bc.add(mb); print(bc)

root:          📤 0x7e04e96571...ee3
hash:          🕟 0xf9a0ba7c86...223
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          1617571974.650272 eth
volume:        42.0 eth
fees:          0.22 eth

txs:
💳 0x4D4d -> 🕗 0xf19d 24.0 eth
🕗 0xf19d -> 💳 0x4D4d 18.0 eth

----------------------------------------

root:          🔛 0xb50e329471...f16
hash:          🔁 0x9b76e4fd28...650
prev_hash:     🕟 0xf9a0ba7c86...223
number:        1
n_txs:         3
mined:         True
time:          1617571992.019669 eth
diff:          2
reward:        50
miner:         👾 0x18a70Cd2EB...c68
nonce:         12
volume:        62.0 eth
fees:          0.21 eth

txs:
👾 0x18a7 -> 👾 0x18a7 50.0 eth
💳 0x4D4d -> 🕗 0xf19d 3.0 eth
🕗 0xf19d -> 💳 0x4D4d 9.0 eth

------
state:
------
💳 0x4D4d017Ea9...444
balance: 24.0
nonce: 2

🕗 0xf19dB46598...b3C
balance: 18.0
nonce: 2

👾 0x18a70Cd2EB...c68
balance: 50.21
nonce: 1



In [9]:
acc4 = Account()
tx1 = acc2.signed_tx(acc4, 12, 1.5)

block = bc.candidate([tx1])
mb    = miner.mine([tx1], mb.header, 2, 50)
bc.add(mb); print(bc)

root:          🔅 0x9f49810139...5c3
hash:          📹 0x93a42646e7...1da
prev_hash:     0x0
number:        0
n_txs:         2
mined:         False
time:          Sat Apr  3 20:46:32 2021
volume:        42.0 eth
fees:          0.22 eth

txs:
📥 0x7f37 -> 🔎 0xa895 24.0 eth
🔎 0xa895 -> 📥 0x7f37 18.0 eth

----------------------------------------

root:          📨 0x820597760a...24f
hash:          💒 0x2c1c14f555...577
prev_hash:     📹 0x93a42646e7...1da
number:        1
n_txs:         3
mined:         True
time:          Sat Apr  3 20:46:34 2021
diff:          2
reward:        50
miner:         📽 0x979Ad81fa9...63D
nonce:         237
volume:        62.0 eth
fees:          0.21 eth

txs:
📽 0x979A -> 📽 0x979A 50.0 eth
📥 0x7f37 -> 🔎 0xa895 3.0 eth
🔎 0xa895 -> 📥 0x7f37 9.0 eth

----------------------------------------

root:          💾 0x58760631ac...260
hash:          🔿 0xd91c1d92d1...332
prev_hash:     💒 0x2c1c14f555...577
number:        2
n_txs:         2
mined:         True
time:          Sat